### Collisional-radiative model for Argon Plasma
#### Rates writing in Plato format
See: https://doi.org/10.1063/1.3585688

In [1]:
import os
import sys
import tqdm
import numpy as np
import dill as pickle

In [2]:
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")
from ronek.plasmas import chem_eq

Inputs

In [3]:
path_to_kin = "./../rates/kin_fit.p"

In [4]:
# Save path
out_path = "./plato/"
os.makedirs(out_path, exist_ok=True)

Load fit rates

In [5]:
kin = pickle.load(open(path_to_kin, "rb"))

Write rates

In [6]:
ARR_PARAM = ("A", "beta", "Ta")

def write_rates(reaction):
  lines = ""
  for indices in tqdm.tqdm(
    iterable=get_indices(reaction),
    desc="> Process '{}'".format(reaction["plato_id"]),
    ncols=80
  ):
    lines += write_rates_line(reaction, indices)
  return lines

def get_indices(reaction):
  indices = np.where(reaction["values"]["A"] > 0.0)
  indices = np.stack(indices,-1).tolist()
  return indices

def write_rates_line(reaction, indices):
  line = ""
  shape = reaction["values"]["A"].shape
  # Write species
  s = 0
  for side in ("reactants", "products"):
    if (side == "products"):
      line = line[:-1] + "="
    for specie in reaction["param"][side]:
      isp = ""
      stoyk_coeff, name, level = specie
      for _ in range(stoyk_coeff):
        isp += name
        if (shape[s] > 1):
          isp += f"({indices[s]+1})"
        else:
          if (name != "em"):
            isp += f"({level+1})"
        isp += "+"
      s += 1
      line += isp[:-1] + "+"
  line = line[:-1]
  # Write rate parameters
  line += ":" + write_rate_coeff(reaction, indices)
  # Write process
  line += ",%d" % reaction["plato_id"]
  # Return line
  return line + "\n"

def write_rate_coeff(reaction, indices):
  string = ""
  for p in ARR_PARAM:
    value = reaction["values"][p][tuple(indices)]
    string += "%.12E," % value
  return string[:-1]

In [7]:
filename = "/home/zanardi/Workspace/AirDatabase/ThermoChemModels/MT/database/kinetics/"

In [8]:
text = "Units=m^3/s\n"
# for k in ("EXh", "Ih", "Ie"):
for k in ("EXh", "EXe", "Ih", "Ie"):
  kin[k]["param"] = chem_eq.get_param(kin[k]["equation"])
  text += write_rates(kin[k])
# with open(out_path + "/argon_kapper", 'w') as f:
with open(filename + "/argon_kapper", 'w') as f:
  f.write(text)

> Process '9': 100%|█████████████████████████| 31/31 [00:00<00:00, 98801.99it/s]


In [9]:
Q11_fit = kin["EN"]["Q11_fit"].reshape(1,-1)
np.savetxt(out_path + "/Q11_em_Ar", Q11_fit, fmt="%.10e")